# Find the top10 regular customers (having atleast one purchase in any month) who are eligible for a special offer. Also demonstrate the performance gains achieved by using persist.

In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config('spark.sql.warehouse.dir',f'/user/{username}/warehouse'). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [3]:
cust_schema = 'cust_id long, purchase_date date, product_id long, amount double'
cust_df = spark.read. \
format('csv'). \
schema(cust_schema). \
load('/public/trendytech/datasets/cust_transf.csv')

In [4]:
cust_df.show()

+-------+-------------+----------+------+
|cust_id|purchase_date|product_id|amount|
+-------+-------------+----------+------+
|   1001|   2023-05-15|      1001| 49.99|
|   1002|   2023-05-16|      1002| 29.99|
|   1003|   2023-05-17|      1003| 39.99|
|   1004|   2023-05-18|      1004| 19.99|
|   1005|   2023-05-19|      1005| 24.99|
|   1001|   2023-05-20|      1002| 29.99|
|   1002|   2023-05-21|      1003| 39.99|
|   1003|   2023-05-22|      1004| 19.99|
|   1004|   2023-05-23|      1005| 24.99|
|   1005|   2023-05-24|      1001| 49.99|
|   1001|   2023-05-25|      1003| 39.99|
|   1002|   2023-05-26|      1004| 19.99|
|   1003|   2023-05-27|      1005| 24.99|
|   1004|   2023-05-28|      1001| 49.99|
|   1005|   2023-05-29|      1002| 29.99|
|   1001|   2023-05-30|      1003| 39.99|
|   1002|   2023-05-31|      1004| 19.99|
|   1003|   2023-06-01|      1005| 24.99|
|   1004|   2023-06-02|      1001| 49.99|
|   1005|   2023-06-03|      1002| 29.99|
+-------+-------------+----------+

In [9]:
from pyspark.sql.functions import year, month
new_df = cust_df.withColumn("purchase_year", year("purchase_date")).withColumn("purchase_month", month("purchase_date"))

In [15]:
from pyspark.sql.functions import countDistinct
customer_month_counts = new_df.groupBy("cust_id","purchase_year","purchase_month").agg(countDistinct("purchase_month").alias("distinct_months"))

In [17]:
regular_customer = customer_month_counts.filter("distinct_months = 1").groupBy("custom_id").count() \
.orderBy("count", ascending=False).limit(10).show()

AnalysisException: "cannot resolve '`customer_id`' given input columns: [cust_id, purchase_year, purchase_month, distinct_months];;\n'Aggregate ['customer_id], ['customer_id, count(1) AS count#90L]\n+- Filter (distinct_months#77L = cast(1 as bigint))\n   +- Aggregate [cust_id#0L, purchase_year#31, purchase_month#37], [cust_id#0L, purchase_year#31, purchase_month#37, count(distinct purchase_month#37) AS distinct_months#77L]\n      +- Project [cust_id#0L, purchase_date#1, product_id#2L, amount#3, purchase_year#31, month(purchase_date#1) AS purchase_month#37]\n         +- Project [cust_id#0L, purchase_date#1, product_id#2L, amount#3, year(purchase_date#1) AS purchase_year#31]\n            +- Relation[cust_id#0L,purchase_date#1,product_id#2L,amount#3] csv\n"